Code to extract and save $\Delta F / F$ for whole making whole brain maps

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import copy
import glob
from pathlib import Path
import pickle
#import re

#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import janelia_core
from janelia_core.dataprocessing.dataset import ROIDataset
#from janelia_core.utils.data_saving import append_ts

#from keller_zlatic_vnc.data_processing import extract_transitions
from keller_zlatic_vnc.data_processing import calc_dff
from keller_zlatic_vnc.data_processing import match_annotation_subject_to_volume_subject
from keller_zlatic_vnc.data_processing import read_raw_transitions_from_excel
#from keller_zlatic_vnc.data_processing import recode_beh
from keller_zlatic_vnc.data_processing import down_select_events
from keller_zlatic_vnc.data_processing import find_before_and_after_events
from keller_zlatic_vnc.data_processing import generate_standard_id_for_full_annots
from keller_zlatic_vnc.data_processing import read_full_annotations
from keller_zlatic_vnc.data_processing import whole_brain_extract_dff_with_annotations

\\keller-s13\shared\exchange\will\bishoplab\projects\keller_drive\janelia_core\janelia_core\fileio\exp_reader.py:21: UserWarning: Unable to locate pyklb module.  Will not be able to read in .klb files.
  warnings.warn('Unable to locate pyklb module.  Will not be able to read in .klb files.')


## Parameters go here

In [3]:
ps = dict()

# The file specifying which subjects we should include in the analysis
ps['subject_file'] = r'Z:\Exchange\Will\bishoplab\projects\keller_drive\keller_vnc\data\single_cell\subjects.csv'

# Location of folders containing annotations
ps['a4_annot_folder'] = r'Z:\Exchange\Will\bishoplab\projects\keller_drive\keller_vnc\data\full_annotations\behavior_csv_cl_A4'
ps['a9_annot_folder'] = r'Z:\Exchange\Will\bishoplab\projects\keller_drive\keller_vnc\data\full_annotations\behavior_csv_cl_A9'

# Location of file containing Chen's annotations - we use this to filter down to only good stimulus events
ps['chen_file'] = r'Z:\Exchange\Will\bishoplab\projects\keller_drive\keller_vnc\data\extracted_dff_v2\transition_list_CW_11202021.xlsx'

# Location of excel file specifying where the data for each experiment is saved relative to the base folder
ps['data_loc_file'] = r'Z:\Exchange\Will\bishoplab\projects\keller_drive\keller_vnc\data\experiment_data_locations.xlsx'

# Base folder where datasets are stored 
ps['dataset_base_folder'] =r'W:\\SV4'

# Subfolder containing the dataset for each subject
ps['dataset_folder'] = 'extracted'

# Parameters for declaring preceeding and succeeding quiet behaviors
ps['pre_q_th'] = 50
ps['succ_q_th'] = 9

# Parameters for determing the location of the marked preceeding and succeeding quiet events
ps['pre_q_event_l'] = 3 # Event length for the preceeding quiet event
ps['succ_q_event_l'] = 3 # Event length for the succeeding quiet event

# Data to calculate Delta F/F for in each dataset
ps['f_ts_str'] = 'f_5_25_25'
ps['bl_ts_str'] = 'bl_5_25_25_long'

# Parameters for calculating Delta F/F
ps['dff_calc_params'] = dict()
ps['dff_calc_params']['background'] = 100
ps['dff_calc_params']['ep'] = 20


# Specify where we save our results
test_type = 'decision_dependence'
ps['save_folder'] = r'Z:\Exchange\Will\bishoplab\projects\keller_drive\keller_vnc\results\new_whole_brain_stats\5_25_25'
ps['save_str'] = test_type + '_5_25_25'



if test_type == 'after_reporting':
    # The type of window we are aligning
    ps['dff_window_type'] = 'start_aligned'

    # The reference we use for aligning windows
    ps['dff_window_ref'] = 'beh_after_start'

    # The offset we applying when placing DFF windows
    ps['dff_window_offset'] = 0

    # The length of the window we calculate DFF in
    ps['dff_window_length'] = 3

    # The event we align the end of the window to (if not using windows of fixed length)
    ps['dff_window_end_ref'] = None

    # The offset we use when locating the end of the window (if not using windows of fixed length)
    ps['dff_window_end_offset'] = None

elif test_type == 'before_reporting':
    ps['dff_window_type'] = 'start_aligned'
    ps['dff_window_ref'] = 'beh_before_start'
    ps['dff_window_offset'] = 0
    ps['dff_window_length'] = 3
    ps['dff_window_end_ref'] = None
    ps['dff_window_end_offset'] = None
elif test_type == 'prediction_dependence':
    ps['dff_window_type'] = 'start_aligned'
    ps['dff_window_ref'] = 'start'
    ps['dff_window_offset'] = -3
    ps['dff_window_length'] = 3
    ps['dff_window_end_ref'] = None
    ps['dff_window_end_offset'] = None
elif test_type == 'state_dependence':
    ps['dff_window_type'] = 'start_aligned'
    ps['dff_window_ref'] = 'beh_after_start'
    ps['dff_window_offset'] = 0
    ps['dff_window_length'] = 3
    ps['dff_window_end_ref'] = None
    ps['dff_window_end_offset'] = None
elif test_type == 'decision_dependence':
    ps['dff_window_type'] = 'start_end_aligned'
    ps['dff_window_ref'] = 'start'
    ps['dff_window_offset'] = 0
    ps['dff_window_length'] = None
    ps['dff_window_end_ref'] = 'end'
    ps['dff_window_end_offset'] = 1 # +1 to include the last point of stimulation
else:
    raise(ValueError('The test type ' + ps['test_type'] + ' is not recognized.'))


## Read in excel file specifying location of datasets

In [4]:
def c_fcn(str):
    return str.replace("'", "")
converters = {0:c_fcn, 1:c_fcn}

data_locs = pd.read_excel(ps['data_loc_file'], header=1, usecols=[1, 2], converters=converters)

## Find stimulus events for all subjects

In [5]:
# Get list of subjects we have annotations for

a4_file_paths = glob.glob(str(Path(ps['a4_annot_folder']) / '*.csv'))
a9_file_paths = glob.glob(str(Path(ps['a9_annot_folder']) / '*.csv'))

n_annot_files = len(a4_file_paths) + len(a9_file_paths)
a4_files = np.zeros(n_annot_files, dtype=bool)
a4_files[0:len(a4_file_paths)] = True

annot_file_paths = a4_file_paths + a9_file_paths

annot_file_names = [Path(p).name for p in annot_file_paths]
annot_subjs = [generate_standard_id_for_full_annots(fn) for fn in annot_file_names]


## Get stimulus events for each subject

In [6]:
subj_events = pd.DataFrame()

for ind, subj in enumerate(annot_subjs):

    # Load the annotations for this subject
    tbl = read_full_annotations(annot_file_paths[ind])

    # Pull out stimulus events for this subject, noting what comes before and after
    stim_tbl = copy.deepcopy(tbl[tbl['beh'] == 'S'])
    stim_tbl.insert(0, 'subject_id', subj)
    stim_tbl.insert(1, 'event_id', range(stim_tbl.shape[0]))
    if a4_files[ind] == True:
        stim_tbl.insert(2, 'manipulation_tgt', 'A4')
    else:
        stim_tbl.insert(2, 'manipulation_tgt', 'A9')
    before_after_tbl = find_before_and_after_events(events=stim_tbl, all_events=tbl)
    stim_annots = pd.concat([stim_tbl, before_after_tbl], axis=1)
    subj_events = subj_events.append(stim_annots, ignore_index=True)


C:\Users\SiMView\AppData\Local\Temp\ipykernel_15000\1346567246.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subj_events = subj_events.append(stim_annots, ignore_index=True)
C:\Users\SiMView\AppData\Local\Temp\ipykernel_15000\1346567246.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subj_events = subj_events.append(stim_annots, ignore_index=True)
C:\Users\SiMView\AppData\Local\Temp\ipykernel_15000\1346567246.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subj_events = subj_events.append(stim_annots, ignore_index=True)
C:\Users\SiMView\AppData\Local\Temp\ipykernel_15000\1346567246.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

## Get rid of any events where we could not classify the type of preceeding or succeeding behavior

In [7]:
subj_events = subj_events.dropna()

## Get rid of any stimulus events which are not also in Chen's annotations

We do this because some of the stimulus events in the full annotations (Nadine's annotations) should be removed because of artefacts. Chen's annotations only include the stimulus events we should analyze.

In [8]:
chen_events = read_raw_transitions_from_excel(file=ps['chen_file'])
chen_events = chen_events.rename(columns={'Manipulation Start': 'start', 'Manipulation End': 'end'})
subj_events = down_select_events(tbl_1=subj_events, tbl_2=chen_events)

Removing event from: 4359-4364 for subject CW_17-08-23-L4
Removing event from: 6004-6009 for subject CW_17-08-23-L4
Removing event from: 267-272 for subject CW_17-09-01-L3


## Mark preceeding and succeeding quiet events

In [9]:
delta_before = subj_events['start'] - subj_events['beh_before_end']
delta_after = subj_events['beh_after_start'] - subj_events['end']

before_quiet_inds = delta_before > ps['pre_q_th']
after_quiet_inds = delta_after > ps['succ_q_th']

subj_events.loc[before_quiet_inds, 'beh_before'] = 'Q'
subj_events.loc[after_quiet_inds, 'beh_after'] = 'Q'


## Mark the start and stop of the marked quiet events

In [10]:
new_before_start = (np.ceil((subj_events[before_quiet_inds]['start'] -
                             subj_events[before_quiet_inds]['beh_before_end']) / 2) +
                             subj_events[before_quiet_inds]['beh_before_end'])

new_before_end = new_before_start + ps['pre_q_event_l'] - 1  # Minus 1 for inclusive indexing
subj_events.loc[before_quiet_inds, 'beh_before_start'] = new_before_start
subj_events.loc[before_quiet_inds, 'beh_before_end'] = new_before_end

new_after_start = (np.ceil((subj_events[after_quiet_inds]['beh_after_start'] -
                            subj_events[after_quiet_inds]['end']) / 2) +
                            subj_events[after_quiet_inds]['end'])
new_after_end = new_after_start + ps['succ_q_event_l'] - 1  # Minus 1 for inclusive indexing
subj_events.loc[after_quiet_inds, 'beh_after_start'] = new_after_start
subj_events.loc[after_quiet_inds, 'beh_after_end'] = new_after_end

## Extract dff along with behavior information from each dataset

In [11]:
analysis_subjects = subj_events['subject_id'].unique()
n_datasets = len(data_locs)
n_analysis_subjs = len(analysis_subjects)
matched_analysis_subjs = np.zeros(n_analysis_subjs)
matched_data_subjs = np.zeros(n_datasets)

subject_event_data = [None]*n_datasets

for d_i in range(n_datasets):
    
    # ===============================================================================
    # Find the appropriate annotations for this experiment
    data_main_folder = data_locs['Main folder'][d_i]
    data_sub_folder = data_locs['Subfolder'][d_i]

    # Parse the subject from the subfolder string
    match_ind = match_annotation_subject_to_volume_subject(data_main_folder, data_sub_folder, analysis_subjects)
    
    if match_ind is not None:
        match_ind = match_ind.item()
        
        
        # Prevent any double matching
        if matched_analysis_subjs[match_ind] == True:
            raise(RuntimeError('Found a transition subject we already matched to: d_i=' + str(d_i)))
        if matched_data_subjs[d_i] == True:
            raise(RuntimeError('Found a volume subject we already matched to.'))
            
        matched_analysis_subjs[match_ind] = True
        matched_data_subjs[d_i] = True
         
        # ===============================================================================
        # Now that we know we have a unique match, we do the rest of our processing

        # Read in the data for this subject
        dataset_path = (Path(ps['dataset_base_folder']) / data_main_folder / data_sub_folder / 
                        Path(ps['dataset_folder']))
        dataset_file = glob.glob(str(dataset_path/'*.pkl'))[0]

        # Load the dataset
        with open(dataset_file, 'rb') as f:
            dataset = ROIDataset.from_dict(pickle.load(f))
        
        # Update the NDArrayHandlers within the dataset object, so they know where to load data from
        for ts_label in dataset.ts_data.keys():
            ts_vls = dataset.ts_data[ts_label]['vls']
            if isinstance(ts_vls, janelia_core.fileio.data_handlers.NDArrayHandler):
                ts_vls.folder = str(dataset_path / Path(ts_vls.folder).name)
        
        # Calculate dff
        f=dataset.ts_data[ps['f_ts_str']]['vls'][:]
        b=dataset.ts_data[ps['bl_ts_str']]['vls'][:]
        dff = calc_dff(f=f, b=b, background=ps['dff_calc_params']['background'], ep=ps['dff_calc_params']['ep'])
        
        # Extract dff for each event
        sample_id = analysis_subjects[match_ind]
        event_rows = subj_events['subject_id'] == sample_id
        sample_events = copy.deepcopy(subj_events[event_rows])
        
        subject_event_data[d_i] = whole_brain_extract_dff_with_annotations(dff=dff,
                                                              event_tbl=sample_events,
                                                              align_col=ps['dff_window_ref'],
                                                              window_type=ps['dff_window_type'],
                                                              ref_offset=ps['dff_window_offset'],
                                                              window_l=ps['dff_window_length'],
                                                              end_align_col=ps['dff_window_end_ref'],
                                                              end_ref_offset=ps['dff_window_end_offset'])
        
        # Give user some feedback
        print('Done processing dataset ' + str(d_i + 1) + ' of ' + str(n_datasets) + '.')

\\keller-s13\shared\exchange\will\bishoplab\projects\keller_drive\keller_vnc\code\keller_zlatic_vnc\keller_zlatic_vnc\data_processing.py:1225: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_tbl = full_tbl.append(row_dict, ignore_index=True)
\\keller-s13\shared\exchange\will\bishoplab\projects\keller_drive\keller_vnc\code\keller_zlatic_vnc\keller_zlatic_vnc\data_processing.py:1225: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_tbl = full_tbl.append(row_dict, ignore_index=True)


Done processing dataset 1 of 65.


\\keller-s13\shared\exchange\will\bishoplab\projects\keller_drive\keller_vnc\code\keller_zlatic_vnc\keller_zlatic_vnc\data_processing.py:1225: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_tbl = full_tbl.append(row_dict, ignore_index=True)
\\keller-s13\shared\exchange\will\bishoplab\projects\keller_drive\keller_vnc\code\keller_zlatic_vnc\keller_zlatic_vnc\data_processing.py:1225: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_tbl = full_tbl.append(row_dict, ignore_index=True)
\\keller-s13\shared\exchange\will\bishoplab\projects\keller_drive\keller_vnc\code\keller_zlatic_vnc\keller_zlatic_vnc\data_processing.py:1225: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_tbl = full_tbl.append(row_dict, ignore_index=True

Done processing dataset 2 of 65.


\\keller-s13\shared\exchange\will\bishoplab\projects\keller_drive\keller_vnc\code\keller_zlatic_vnc\keller_zlatic_vnc\data_processing.py:1225: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_tbl = full_tbl.append(row_dict, ignore_index=True)
\\keller-s13\shared\exchange\will\bishoplab\projects\keller_drive\keller_vnc\code\keller_zlatic_vnc\keller_zlatic_vnc\data_processing.py:1225: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_tbl = full_tbl.append(row_dict, ignore_index=True)
\\keller-s13\shared\exchange\will\bishoplab\projects\keller_drive\keller_vnc\code\keller_zlatic_vnc\keller_zlatic_vnc\data_processing.py:1225: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_tbl = full_tbl.append(row_dict, ignore_index=True

...

\\keller-s13\shared\exchange\will\bishoplab\projects\keller_drive\keller_vnc\code\keller_zlatic_vnc\keller_zlatic_vnc\data_processing.py:1225: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_tbl = full_tbl.append(row_dict, ignore_index=True)
\\keller-s13\shared\exchange\will\bishoplab\projects\keller_drive\keller_vnc\code\keller_zlatic_vnc\keller_zlatic_vnc\data_processing.py:1225: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_tbl = full_tbl.append(row_dict, ignore_index=True)
\\keller-s13\shared\exchange\will\bishoplab\projects\keller_drive\keller_vnc\code\keller_zlatic_vnc\keller_zlatic_vnc\data_processing.py:1225: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_tbl = full_tbl.append(row_dict, ignore_index=True

Done processing dataset 63 of 65.


\\keller-s13\shared\exchange\will\bishoplab\projects\keller_drive\keller_vnc\code\keller_zlatic_vnc\keller_zlatic_vnc\data_processing.py:1225: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_tbl = full_tbl.append(row_dict, ignore_index=True)
\\keller-s13\shared\exchange\will\bishoplab\projects\keller_drive\keller_vnc\code\keller_zlatic_vnc\keller_zlatic_vnc\data_processing.py:1225: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_tbl = full_tbl.append(row_dict, ignore_index=True)
\\keller-s13\shared\exchange\will\bishoplab\projects\keller_drive\keller_vnc\code\keller_zlatic_vnc\keller_zlatic_vnc\data_processing.py:1225: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_tbl = full_tbl.append(row_dict, ignore_index=True

Done processing dataset 64 of 65.
Done processing dataset 65 of 65.


\\keller-s13\shared\exchange\will\bishoplab\projects\keller_drive\keller_vnc\code\keller_zlatic_vnc\keller_zlatic_vnc\data_processing.py:1225: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_tbl = full_tbl.append(row_dict, ignore_index=True)
\\keller-s13\shared\exchange\will\bishoplab\projects\keller_drive\keller_vnc\code\keller_zlatic_vnc\keller_zlatic_vnc\data_processing.py:1225: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_tbl = full_tbl.append(row_dict, ignore_index=True)
\\keller-s13\shared\exchange\will\bishoplab\projects\keller_drive\keller_vnc\code\keller_zlatic_vnc\keller_zlatic_vnc\data_processing.py:1225: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_tbl = full_tbl.append(row_dict, ignore_index=True

In [12]:
subject_event_data = pd.concat(subject_event_data, ignore_index=True)

In [13]:
# See what subjects we have transitions for but no registerd volumes
ignored_analysis_subjs = analysis_subjects[np.logical_not(matched_analysis_subjs)]

# See what subjects we have registered volumes for but no transitions
ignored_vol_subjs = data_locs[np.logical_not(matched_data_subjs)]

In [14]:
## Save results
rs = dict()
rs['ps'] = ps
rs['subject_event_data'] = subject_event_data
rs['ignored_analysis_subjs'] = ignored_analysis_subjs
rs['ignored_vol_subjs'] = ignored_vol_subjs

save_name = ps['save_str'] + '.pkl'
save_path = Path(ps['save_folder']) / save_name
with open(save_path, 'wb') as f:
    pickle.dump(rs, f)
    
print('Saved results to: ' + str(save_path))

Saved results to: Z:\Exchange\Will\bishoplab\projects\keller_drive\keller_vnc\results\new_whole_brain_stats\5_25_25\decision_dependence_5_25_25.pkl


In [15]:
subject_event_data

,subject_id,manipulation_tgt,event_id,beh_before,beh_after,beh,dff
0,CW_17-08-23-L1,A4,0,F,P,S,"[0.51608485, 0.48427287, 0.39645705, 0.3752432..."
1,CW_17-08-23-L1,A4,1,F,P,S,"[0.53802836, 0.50630283, 0.41602746, 0.3865444..."
2,CW_17-08-23-L2,A4,0,F,B,S,"[0.5735277, 0.53321326, 0.40003648, 0.38193583..."
3,CW_17-08-23-L2,A4,1,F,B,S,"[0.5210747, 0.47617674, 0.35408345, 0.32054883..."
4,CW_17-08-23-L2,A4,2,F,B,S,"[0.56831515, 0.52191776, 0.37642595, 0.3361944..."
...,...,...,...,...,...,...,...
184,CW_17-12-11-L3,A4,3,Q,F,S,"[1.9190801, 1.8327557, 1.0687201, 0.8503302, 1..."
185,CW_17-12-11-L3,A4,4,Q,F,S,"[1.9736592, 1.841115, 1.0997047, 0.8724971, 1...."
186,CW_17-12-11-L3,A4,5,Q,F,S,"[1.9726671, 1.8393914, 1.1119825, 0.86963326, ..."
187,CW_17-12-11-L3,A4,6,Q,F,S,"[1.9955611, 1.8517638, 1.1175747, 0.8699227, 1..."


In [16]:
subj_events

,subject_id,event_id,manipulation_tgt,start,end,beh,beh_before,beh_before_start,beh_before_end,beh_after,beh_after_start,beh_after_end
0,CW_17-08-23-L1,0,A4,216,220,S,F,192,215,P,228,236
1,CW_17-08-23-L1,1,A4,1795,1799,S,F,1777,1793,P,1805,1819
2,CW_17-08-23-L2,0,A4,114,119,S,F,110,113,B,120,128
3,CW_17-08-23-L2,1,A4,2471,2476,S,F,2452,2467,B,2477,2485
4,CW_17-08-23-L2,2,A4,4644,4649,S,F,4634,4639,B,4649,4657
...,...,...,...,...,...,...,...,...,...,...,...,...
259,CW_17-11-08-L3,1,A9,1797,1802,S,Q,1709,1711,F,1803,1808
260,CW_17-11-08-L3,2,A9,2695,2700,S,Q,2621,2623,F,2701,2707
261,CW_17-11-08-L3,3,A9,3482,3487,S,Q,3290,3292,F,3488,3494
262,CW_17-11-08-L3,4,A9,4181,4186,S,Q,4012,4014,F,4187,4195
